In [1]:
!pip install openai==1.55.3
!pip install beautifulsoup4==4.12.3
!pip install requests==2.31.0
try:
  import deeplake
except:
  !pip install deeplake==3.9.18
  import deeplake

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 5.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.86.0
    Uninstalling openai-1.86.0:
      Successfully uninstalled openai-1.86.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.13.4
    Uninstalling beautifulsoup4-4.13.4:
      Successfully uninstalled beautifulsoup4-4.13.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.3
    Uninstalling requests-2.32.3:
      Successfully uninstalled requests-2.32.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.31.0 which i

/usr/local/lib/python3.11/dist-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (4.2.8) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [26]:
import openai
from openai import OpenAI
import os
import textwrap
import requests
from bs4 import BeautifulSoup
import re

import spacy
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet
from collections import Counter
import numpy as np
import pandas as pd

import textwrap
try:
  import fitz
except:
  !pip install pymupdf

# for similarity search
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [62]:
# set APIs keys into enviroment variables
os.environ['OPEN_API_KEY'] = 'OPEN_API_KEY'
os.environ['ACTIVELOOP_TOKEN'] = 'ACTIVELOOP_TOKEN'

openai.api_key = os.getenv('OPEN_API_KEY')


# Load the public data for create the retrival store

In [12]:
class DataProcessor():

  def __init__(self, urls, doc_file_path='doc_file_path', save_to='filepath'):
    self.urls = urls
    self.save_to = 'knowledgebase.txt'
    self.doc_file_path = doc_file_path


  def clean_text(self, content):
      content = re.sub(r'\d+', '', content)
      return content

  def retrieve_from_url(self, url):
      response = requests.get(url)
      soap = BeautifulSoup(response.text, 'html.parser')
      content = soap.find('div', {'class': 'mw-parser-output'})

      # Extract and clean the content
      text = content.get_text(separator=' ', strip=True)
      text = self.clean_text(content)
      return text

  def retrieve_from_pdf(self, file_name):
      doc = fitz.open(filename=filename)
      return '\n'.join(page.get_text() for page in doc)

  def process_content(self):
    with open('knowledgebase.txt', 'w') as file:
      for url in self.urls:
        content_from_web = self.retrieve_from_url(url)

      content_from_pdf = self.retrieve_from_pdf(self.doc_file_path)

      knowledge_content = content_from_pdf + '\n' + content_from_web

      file.write(knowledge_content + '\n')

In [15]:
urls = ['https://url1', 'https://url2']

data_processor = DataProcessor(urls)

data_processor.process_content()


# Embedding and vector store

In [16]:
from deeplake.core.vectorstore.deeplake_vectorstore import VectorStore
import deeplake
import deeplake.util

chunk_size = 1000
add_to_vector_store = False

/usr/local/lib/python3.11/dist-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (4.2.8) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [36]:
class VectorStoreBuilder():
  def __init__(self,texts, model_name = 'text-embedding-3-small',add_to_vector_store=False, chunk_size=200,vector_store_path = '<vector_store_path'):
    self.model_name = model_name
    self.texts = 'knowledgebase.txt'
    self.vector_store_path = vector_store_path

  def is_vector_store_exist(self):
    try:
      vector_store = VectorStore(self.vector_store_path)
      print('Vector store exists')
    except:
      print('Vector store does not exists')
      self.add_to_vector_store = True

  def embedding_fn(self):
    if isinstance(self.texts, str):
      texts = [self.texts]
      texts = [text.replace('\n','') for text in texts]
      return [data.embedding for data in openai.embeddings.create(input=texts, model=model).data]

  def store_to_vector_db(self):
    if self.add_to_vector_store:
      with open(self.texts, 'r') as file:
        text = file.read()
        chunk_size = self.chunk_size
        chunked_text = [text[i: i+chunk_size] for i in range(0,len(text), chunk_size)] # This is the straight way to do chunking but it may loose context hence we can do overlapping

    vector_store.add(
        text = chunked_text,
        embedding_function= self.embedding_fn,
        embedding_data= chunked_text,
        metadata= [{'source': 'knowledgebase.txt'}] * len(chunked_text)
)


In [41]:
vector_store_builder = VectorStoreBuilder(texts='knowledgebase.txt')
vector_store_builder.is_vector_store_exist()

Vector store does not exists


# RAG

In [58]:
class RAG():
  def __init__(self, vector_store_path = 'vector_store_path'):
    try:
      self.vector_store = deeplake.load(vector_store_path)
      print("Vector DB exist")
    except:
      print("Vector DB does not exist")
      self.vector_store = deeplake.empty  # create empty but no use

  def build_prompt(self,user_prompt):
    search_results = self.vector_store.search(user_prompt, embedding_function=embedding_fn)
    best_similarity_result = search_results['text'][0]
    return f'{user_prompt} {best_similarity_result}'


In [60]:
rag = RAG(vector_store_path = 'vector_store_path')
agumented_prompt = rag.build_prompt("user_prompt")
text_wrapper = textwrap.TextWrapper(width=100)
print(text_wrapper.fill(agumented_prompt))


Vector DB does not exist


# GPT (Generator)

In [64]:
# call GPT with augumented prompt
client = OpenAI()
def call_gpt(text):
  text_input = ''.join(text)
  prompt = f"Please summarize or elobrate on the following content \n {text_input}"
  try:
    response = client.chat.complentions.create(
        model = 'model_name',
        messages = [
            {'role': 'system', 'content': "You are the AI assistance."},
            {'role': 'assistant', 'content': "Please provide the answer based on input"},
            {'role': 'user', 'content': prompt}
        ],
        temperature = 0.1
    )
    return response.choice[0].message.content.strip()

  except Exception as e:
    return str(e)
